In [2]:
!pip install folium

     |████████████████████████████████| 92kB 5.3MB/s eta 0:00:011


In [8]:
#Let's start with importing the necessary libraries for our project
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library
import numpy as np
print('Required Libraries imported...')

Required Libraries imported...


In [29]:
# Foursquare Details
CLIENT_ID = 'xxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxxxxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version



LIMIT = 500 # Maximum is 100

In [5]:
# Top 6 Metro Cities in India
cities = ["Mumbai, IN", 'Chennai, IN', 'Delhi, IN', 'Bangalore, IN', 'Pune, IN','Hyderabad, IN']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d145941735") # Chinese Food CATEGORY ID
    results[city] = requests.get(url).json()

In [6]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [9]:
maps={}    
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Chinese Restaurants in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of Chinese Restaurants in Mumbai, IN =  197
Showing Top 100
Total number of Chinese Restaurants in Chennai, IN =  91
Showing Top 100
Total number of Chinese Restaurants in Delhi, IN =  92
Showing Top 100
Total number of Chinese Restaurants in Bangalore, IN =  174
Showing Top 100
Total number of Chinese Restaurants in Pune, IN =  95
Showing Top 100
Total number of Chinese Restaurants in Hyderabad, IN =  67
Showing Top 100


In [11]:
#Lets look at Mumbai City Map
maps[cities[0]]

In [12]:
#Lets look at Chennai City Map
maps[cities[1]]

In [13]:
#Lets look at Delhi City Map
maps[cities[2]]

In [14]:
#Lets look at Bangalore City Map
maps[cities[3]]

In [15]:
#Lets look at Pune City Map
maps[cities[4]]

In [16]:
#Lets look at Hyderabad City Map
maps[cities[5]]

Here after analysing each metro city in India we can conclude that Mumbai and Bangalore Cities are the most dense cities with Chinese restaurants as food category.

However, Let's have a concrete measure of this density.

For this I will use some basic statistics. I will get the mean location of the Chinese food category places which should be near to most of them if they are really dense or far if not.

Next I will take the average of the distance of the venues to the mean coordinates.

In [21]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="red", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))


Mumbai, IN
Mean Distance from Mean coordinates
0.073579086275835
Chennai, IN
Mean Distance from Mean coordinates
0.04486836549643752
Delhi, IN
Mean Distance from Mean coordinates
0.06398000017914919
Bangalore, IN
Mean Distance from Mean coordinates
0.040402975946871755
Pune, IN
Mean Distance from Mean coordinates
0.05686352044921111
Hyderabad, IN
Mean Distance from Mean coordinates
0.05115826459931344


Lets look at each metro city map to view the mean distance from mean coordinates

In [22]:
#Mumbai City:
maps[cities[0]]

In [23]:
#Chennai City:
maps[cities[1]]

In [24]:
#Delhi City:
maps[cities[2]]

In [25]:
#Bangalore City:
maps[cities[3]]

In [26]:
#Pube City:
maps[cities[4]]

In [27]:
#Hyderabad City:
maps[cities[5]]

As per the above analysis we can now conclude that among all the metro cities in India, Mumbai is the first best option and the second best is Bangalore and Third best is Pune. Any tourist's best interest would be to book a hotel near that mean coordinate to experience chinese food restaurants